# 3. Inference trên ảnh mẫu

Notebook này chạy inference trên các ảnh mẫu để test hệ thống VQA.


## 1. Cài đặt thư viện


In [1]:
%pip install -q transformers accelerate bitsandbytes qwen-vl-utils sentence-transformers rank-bm25 underthesea wikipedia pillow pandas tqdm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 89.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 52.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# token = user_secrets.get_secret("HF_TOKEN")

# login(token)

## 2. Copy source code vào working directory


In [3]:
import shutil
from pathlib import Path

# Copy source code to Kaggle working directory
if Path('/kaggle/working').exists():
    # Option 1: Copy from input dataset (if you uploaded code as dataset)
    code_source = Path('/kaggle/input') / 'ravqa-code' / 'code'  # Adjust path to your dataset
    code_dest = Path('/kaggle/working/code')
    
    if code_source.exists():
        if code_dest.exists():
            shutil.rmtree(code_dest)
        shutil.copytree(code_source, code_dest)
        print(f" Code copied to {code_dest}")
    else:
        print(f"  Code source not found at {code_source}")
        print("Creating src directory - you'll need to add your source files manually")
        # Create src directory structure manually
        code_dest = Path('/kaggle/working/code/src')
        code_dest.mkdir(parents=True, exist_ok=True)
        print(f"Created {code_dest}")
        print("Please add your source files (pipeline.py, vision.py, etc.) to this directory")
else:
    print("Not running in Kaggle environment")
    code_dest = Path('../src')


 Code copied to /kaggle/working/code


## 3. Tạo source code files (nếu chưa có trong dataset)


In [4]:
# Nếu bạn chưa upload code lên Kaggle dataset, bạn có thể tạo các file cần thiết ở đây
# Hoặc copy từ input dataset nếu đã upload

from pathlib import Path
import os

code_dir = Path('/kaggle/working/code/src') if Path('/kaggle/working').exists() else Path('../src')
code_dir.mkdir(parents=True, exist_ok=True)

# Kiểm tra xem đã có file chưa
if (code_dir / 'pipeline.py').exists():
    print(" Source files already exist")
else:
    print("  Source files not found. Please:")
    print("1. Upload your code folder as a Kaggle dataset, OR")
    print("2. Manually copy pipeline.py, vision.py, answering.py, retrieval.py, config.py to:")
    print(f"   {code_dir}")
    print("\nYou can also use the 'Add Data' feature in Kaggle to upload your code folder.")


 Source files already exist


## 4. Import thư viện và setup paths


In [5]:
pwd

'/kaggle/working'

In [6]:
import json
import sys
from pathlib import Path
from PIL import Image
import pandas as pd
from tqdm import tqdm
import torch
import os

# Add src to path
if Path('/kaggle/working').exists():
    sys.path.insert(0, '/kaggle/working/code/src')
else:
    sys.path.insert(0, '../src')
    
sys.path.insert(0, '/kaggle/working/code')


In [7]:
# Import pipeline
try:
    from src.pipeline import RAGVQAPipeline
    print(" Pipeline imported successfully")
except ImportError as e:
    print(f" Error importing pipeline: {e}")
    print("Please make sure source files are in the correct location")
    raise

2025-12-29 17:27:35.646371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767029255.863406     108 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767029255.928917     108 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767029256.465793     108 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767029256.465835     108 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767029256.465838     108 computation_placer.cc:177] computation placer alr

 Pipeline imported successfully


## 5. Cấu hình đường dẫn


In [8]:
# Paths cho Kaggle
if Path('/kaggle/working').exists():
    # Kaggle paths - ĐIỀU CHỈNH THEO DATASET CỦA BẠN
    KB_PATH = "/kaggle/input/knowledge-base/knowledge_base.json"
    IMAGES_DIR = "/kaggle/input/vqa-dataset/images_flat/images_flat"
    VECTOR_DB_PATH = "/kaggle/input/vector-db"  # Nếu đã build index trước đó
    OUTPUT_DIR = "/kaggle/working/results"
else:
    # Local paths
    KB_PATH = "../../data/knowledge_base.json"
    IMAGES_DIR = "../../data/images_flat"
    VECTOR_DB_PATH = "../../models/vector_db"
    OUTPUT_DIR = "../../results"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Sample images to test - THAY ĐỔI THEO TÊN ẢNH THỰC TẾ
SAMPLE_IMAGES = [
    "004122.jpg",
    "004129.jpg",
    "004140.jpg",
]

# Sample questions
SAMPLE_QUESTIONS = [
    "Đây là gì? Hãy mô tả chi tiết hình ảnh này.",
    "Hãy giải thích chi tiết về hình ảnh này.",
    "Thông tin trong hình ảnh này nói về gì?"
]

print(f"Knowledge Base: {KB_PATH}")
print(f"Images Directory: {IMAGES_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"\nWill process {len(SAMPLE_IMAGES)} sample images")


Knowledge Base: /kaggle/input/knowledge-base/knowledge_base.json
Images Directory: /kaggle/input/vqa-dataset/images_flat/images_flat
Output Directory: /kaggle/working/results

Will process 3 sample images


## 6. Khởi tạo Pipeline


In [9]:
print("Initializing pipeline...")
print("This may take a few minutes to load the models...")
print("(Downloading Qwen2VL-7B model if not cached)")

# Initialize pipeline with 4-bit quantization for T4 GPU
pipeline = RAGVQAPipeline(
    vision_model_name="Qwen/Qwen2-VL-7B-Instruct",
    answering_model_name="Qwen/Qwen2-VL-7B-Instruct",
    use_4bit=True
)

print(" Pipeline ready!")


Initializing pipeline...
This may take a few minutes to load the models...
(Downloading Qwen2VL-7B model if not cached)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

 Pipeline ready!


## 7. Chạy inference trên ảnh mẫu


In [10]:
results = []

for i, image_name in enumerate(tqdm(SAMPLE_IMAGES, desc="Processing images")):
    try:
        # Load image
        image_path = Path(IMAGES_DIR) / image_name
        
        if not image_path.exists():
            print(f"  Image not found: {image_path}")
            continue
        
        image = Image.open(image_path).convert('RGB')
        
        # Use corresponding question or default
        question = SAMPLE_QUESTIONS[i] if i < len(SAMPLE_QUESTIONS) else SAMPLE_QUESTIONS[0]
        
        print(f"\n{'='*80}")
        print(f"Image: {image_name}")
        print(f"Question: {question}")
        print(f"{'='*80}")
        
        # Get prediction
        result = pipeline.process(
            image=image,
            question=question,
            return_intermediate=True
        )
        
        # Separate retrieved docs by source
        retrieved_docs = result.get('retrieved_docs', [])
        kb_docs = [doc for doc in retrieved_docs if doc.get('source', '').startswith('kb_')]
        wiki_docs = [doc for doc in retrieved_docs if doc.get('source') == 'wikipedia']
        
        # Extract Wikipedia search keywords (if any)
        wiki_keywords = None
        if wiki_docs and len(wiki_docs) > 0:
            wiki_keywords = wiki_docs[0].get('keywords_used', None)
        
        # Store results
        results.append({
            'image': image_name,
            'question': question,
            'answer': result['answer'],
            'caption': result.get('caption', ''),
            'ocr': result.get('ocr', ''),
            'num_retrieved': len(retrieved_docs),
            'num_kb_docs': len(kb_docs),
            'num_wiki_docs': len(wiki_docs),
            'wiki_keywords': wiki_keywords,
            'context': result.get('context', '')[:500]  # Truncate for display
        })
        
        # Display result
        print(f"\n CAPTION:")
        print(f"{result.get('caption', '')[:300]}..." if len(result.get('caption', '')) > 300 else result.get('caption', ''))
        
        print(f"\n OCR:")
        print(f"{result.get('ocr', '')[:200] if result.get('ocr') else 'None'}...")
        
        print(f"\n RETRIEVAL RESULTS:")
        print(f"  Total retrieved: {len(retrieved_docs)}")
        print(f"  - From Knowledge Base: {len(kb_docs)}")
        print(f"  - From Wikipedia: {len(wiki_docs)}")
        
        # Display Wikipedia search keywords
        if wiki_keywords:
            print(f"\n Wikipedia Search Keywords (VLM-generated):")
            print(f"  {wiki_keywords}")
        
        # Display Knowledge Base passages
        if kb_docs:
            print(f"\n KNOWLEDGE BASE PASSAGES ({len(kb_docs)}):")
            for j, doc in enumerate(kb_docs, 1):
                print(f"\n  [{j}] Source: {doc.get('source', 'unknown')}")
                if 'score' in doc:
                    print(f"      Score: {doc['score']:.4f}")
                if 'entity' in doc:
                    print(f"      Entity: {doc.get('entity', 'N/A')}")
                if 'summary' in doc:
                    summary = doc.get('summary', '')
                    print(f"      Summary: {summary[:200]}..." if len(summary) > 200 else f"      Summary: {summary}")
                if 'facts' in doc:
                    facts = doc.get('facts', '')
                    if facts:
                        print(f"      Facts: {facts[:200]}..." if len(facts) > 200 else f"      Facts: {facts}")
        
        # Display Wikipedia passages
        if wiki_docs:
            print(f"\n WIKIPEDIA PASSAGES ({len(wiki_docs)}):")
            for j, doc in enumerate(wiki_docs, 1):
                print(f"\n  [{j}] Title: {doc.get('title', 'N/A')}")
                if 'url' in doc:
                    print(f"      URL: {doc.get('url', 'N/A')}")
                if 'keywords_used' in doc:
                    print(f"      Search Keywords: {doc.get('keywords_used', 'N/A')}")
                if 'content' in doc:
                    content = doc.get('content', '')
                    print(f"      Content: {content[:300]}..." if len(content) > 300 else f"      Content: {content}")
        
        print(f"\n FINAL ANSWER:")
        print(result['answer'])
        
    except Exception as e:
        print(f" Error processing {image_name}: {e}")
        import traceback
        traceback.print_exc()
        results.append({
            'image': image_name,
            'question': question if 'question' in locals() else '',
            'answer': f'ERROR: {str(e)}',
            'caption': '',
            'ocr': '',
            'num_retrieved': 0,
            'num_kb_docs': 0,
            'num_wiki_docs': 0,
            'wiki_keywords': None,
            'context': ''
        })

print(f"\n Completed inference on {len(results)} images")


Processing images:   0%|          | 0/3 [00:00<?, ?it/s]


Image: 004122.jpg
Question: Đây là gì? Hãy mô tả chi tiết hình ảnh này.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
Processing images:  33%|███▎      | 1/3 [00:43<01:27, 43.93s/it]


 CAPTION:
Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc ...

 OCR:
None...

 RETRIEVAL RESULTS:
  Total retrieved: 3
  - From Knowledge Base: 0
  - From Wikipedia: 3

 Wikipedia Search Keywords (VLM-generated):
  Cậu bé, xe đạp đồ chơi, con báo, công viên, cây cầu

 WIKIPEDIA PASSAGES (3):

  [1] Title: Danh sách tập phim Shin – Cậu bé bút chì (2022–nay)
      URL: https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_nh%C3%A2n_v%E1%BA%ADt_trong_Shin_%E2%80%93_C%E1%BA%ADu_b%C3%A9_b%C3%BAt_ch%C3%AC
      Search Keywords: Cậu bé, xe đạp đồ chơi, con báo, công viên, cây cầu
      Content: Đây là danh sách nhân vật trong series anime và manga Shin – Cậu bé bút chì được sáng tác bởi Usui Yoshito.
Cậu bé Cu Shin đã góp Phần tạo nên sự vui nhộn và hài

ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
Processing images:  67%|██████▋   | 2/3 [01:23<00:41, 41.50s/it]


 CAPTION:
Hình ảnh này mô tả một chiếc quay đĩa màu nâu, được làm từ vật liệu như đất sét hoặc nhựa. Quay đĩa có hình dạng tròn và mỏng, với một phần trên đầu có hình dạng giống như một nắp. Quay đĩa này đang quay trên một mặt đất có màu xám, có thể là một con đường hoặc một mặt đất mở. Quay đĩa có đường viền...

 OCR:
None...

 RETRIEVAL RESULTS:
  Total retrieved: 0
  - From Knowledge Base: 0
  - From Wikipedia: 0

 FINAL ANSWER:
Hình ảnh này mô tả một chiếc quay đĩa màu nâu, được làm từ vật liệu như đất sét hoặc nhựa. Quay đĩa có hình dạng tròn và mỏng, với một phần trên đầu có hình dạng giống như một nắp. Quay đĩa này đang quay trên một mặt đất có màu xám, có thể là một con đường hoặc một mặt đất mở. Quay đĩa có đường viền màu đen, tạo nên một thiết kế đơn giản nhưng độc đáo.

Quay đĩa là một trò chơi truyền thống của người Việt, được sử dụng từ thời cổ đại. Nó được làm từ nhiều loại vật liệu khác nhau, bao gồm đất sét, nhựa, hoặc gỗ. Quay đĩa thường có hình dạng tròn và mỏng, với

ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
Processing images: 100%|██████████| 3/3 [02:24<00:00, 48.25s/it]


 CAPTION:
Hình ảnh này mô tả một cảnh lễ hội truyền thống ở Việt Nam. Các người trong hình đang mặc trang phục truyền thống, chủ yếu là màu đỏ và vàng, với các họa tiết đặc trưng của văn hóa Việt Nam. Họ đang tham gia vào một nghi lễ hoặc lễ hội, với một số người đang chơi trống và một số khác đang chuẩn bị t...

 OCR:
None...

 RETRIEVAL RESULTS:
  Total retrieved: 3
  - From Knowledge Base: 0
  - From Wikipedia: 3

 Wikipedia Search Keywords (VLM-generated):
  Lễ hội truyền thống, Việt Nam, trang phục truyền thống, văn hóa Việt Nam, nồi nấu

 WIKIPEDIA PASSAGES (3):

  [1] Title: Ẩm thực Việt Nam
      URL: https://vi.wikipedia.org/wiki/%E1%BA%A8m_th%E1%BB%B1c_Vi%E1%BB%87t_Nam
      Search Keywords: Lễ hội truyền thống, Việt Nam, trang phục truyền thống, văn hóa Việt Nam, nồi nấu
      Content: ==== Tiệc tùng ====

Tiệc có nhiều loại, tuy nhiên theo truyền thống thường là một dạng cỗ với nhiều món ăn mặn, nem, rau, nộm, món tráng miệng, và rượu hoặc bia uống kèm. Ngày nay tiệc có th

## 8. Lưu kết quả


In [11]:
# Save to JSON
output_json = f"{OUTPUT_DIR}/inference_results.json"
with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f" Results saved to {output_json}")

# Save to CSV for easy viewing
df = pd.DataFrame(results)
output_csv = f"{OUTPUT_DIR}/inference_results.csv"
df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f" Results saved to {output_csv}")

# Display summary
print("\n=== Inference Summary ===")
print(f"Total images processed: {len(results)}")
if len(results) > 0:
    print(f"Average retrieved docs: {df['num_retrieved'].mean():.2f}")
    print(f"Images with caption: {(df['caption'] != '').sum()}")
    print(f"Images with OCR: {(df['ocr'] != '').sum()}")


 Results saved to /kaggle/working/results/inference_results.json
 Results saved to /kaggle/working/results/inference_results.csv

=== Inference Summary ===
Total images processed: 3
Average retrieved docs: 2.00
Images with caption: 3
Images with OCR: 0


## 9. Hiển thị kết quả chi tiết


In [12]:
# Display all results
for i, result in enumerate(results):
    print(f"\n{'='*80}")
    print(f"Sample {i+1}: {result['image']}")
    print(f"{'='*80}")
    print(f"Question: {result['question']}")
    print(f"\nAnswer:")
    print(result['answer'])
    if result['caption']:
        print(f"\nCaption: {result['caption'][:300]}..." if len(result['caption']) > 300 else f"\nCaption: {result['caption']}")
    if result['ocr']:
        print(f"\nOCR: {result['ocr'][:200]}..." if len(result['ocr']) > 200 else f"\nOCR: {result['ocr']}")
    print(f"\nRetrieved docs: {result['num_retrieved']}")



Sample 1: 004122.jpg
Question: Đây là gì? Hãy mô tả chi tiết hình ảnh này.

Answer:
Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc khu vui chơi trẻ em, với một cây lớn và một cây cầu trong khoảng cách. Cây cầu có nhiều cột và đường đi bộ, tạo nên một cảnh quan mở và thoáng đãng. 

Không có công trình hoặc đặc điểm văn hóa cụ thể được mô tả trong hình ảnh này.

Caption: Hình ảnh này mô tả một cậu bé đang chơi trên một chiếc xe đạp đồ chơi hình con báo. Cậu bé mặc một chiếc áo thun màu đỏ và quần short màu xanh. Trên chiếc xe đạp, có một hình vẽ của con báo với màu vàng và đen, tạo nên một cảm giác vui nhộn và tươi mới. 

Bối cảnh của hình ảnh là một công viên hoặc ...

Retrieved docs: 3

Sample 2: 004129.jpg
Question: Hãy giải thích chi 

## 10. Test với một ảnh và câu hỏi tùy chỉnh


In [13]:
# Test với một ảnh cụ thể - THAY ĐỔI THEO Ý BẠN
test_image_name = "004160.jpg"  # Thay đổi tên ảnh
test_question = "Địa điểm trong ảnh có phải di sản thiên nhiên thế giới không?"

test_image_path = Path(IMAGES_DIR) / test_image_name

if test_image_path.exists():
    test_image = Image.open(test_image_path).convert('RGB')
    
    print(f"{'='*80}")
    print(f"Testing with image: {test_image_name}")
    print(f"Question: {test_question}")
    print(f"{'='*80}")
    print("\n Processing...")
    
    result = pipeline.process(
        image=test_image,
        question=test_question,
        return_intermediate=True
    )
    
    # Separate retrieved docs by source
    retrieved_docs = result.get('retrieved_docs', [])
    kb_docs = [doc for doc in retrieved_docs if doc.get('source', '').startswith('kb_')]
    wiki_docs = [doc for doc in retrieved_docs if doc.get('source') == 'wikipedia']
    
    # Extract Wikipedia search keywords
    wiki_keywords = None
    if wiki_docs and len(wiki_docs) > 0:
        wiki_keywords = wiki_docs[0].get('keywords_used', None)
    
    print(f"\n{'='*80}")
    print(" DETAILED RESULTS")
    print(f"{'='*80}")
    
    print(f"\n CAPTION:")
    print(result.get('caption', ''))
    
    print(f"\n OCR:")
    print(result.get('ocr', 'None'))
    
    print(f"\n RETRIEVAL SUMMARY:")
    print(f"  Total retrieved documents: {len(retrieved_docs)}")
    print(f"  - From Knowledge Base: {len(kb_docs)}")
    print(f"  - From Wikipedia: {len(wiki_docs)}")
    
    # Display Wikipedia search keywords
    if wiki_keywords:
        print(f"\n WIKIPEDIA SEARCH KEYWORDS (VLM-generated):")
        print(f"  {wiki_keywords}")
    elif len(wiki_docs) == 0:
        print(f"\n WIKIPEDIA SEARCH KEYWORDS: Not used (no Wikipedia results)")
    
    # Display Knowledge Base passages in detail
    if kb_docs:
        print(f"\n{'='*80}")
        print(f" KNOWLEDGE BASE PASSAGES ({len(kb_docs)}):")
        print(f"{'='*80}")
        for j, doc in enumerate(kb_docs, 1):
            print(f"\n[{j}] Source: {doc.get('source', 'unknown')}")
            if 'score' in doc:
                print(f"    Score: {doc['score']:.4f}")
            if 'entity' in doc:
                print(f"    Entity: {doc.get('entity', 'N/A')}")
            if 'summary' in doc:
                summary = doc.get('summary', '')
                print(f"    Summary: {summary}")
            if 'facts' in doc:
                facts = doc.get('facts', '')
                if facts:
                    print(f"    Facts: {facts}")
    else:
        print(f"\n KNOWLEDGE BASE PASSAGES: None")
    
    # Display Wikipedia passages in detail
    if wiki_docs:
        print(f"\n{'='*80}")
        print(f" WIKIPEDIA PASSAGES ({len(wiki_docs)}):")
        print(f"{'='*80}")
        for j, doc in enumerate(wiki_docs, 1):
            print(f"\n[{j}] Title: {doc.get('title', 'N/A')}")
            if 'url' in doc:
                print(f"    URL: {doc.get('url', 'N/A')}")
            if 'keywords_used' in doc:
                print(f"    Search Keywords Used: {doc.get('keywords_used', 'N/A')}")
            if 'content' in doc:
                content = doc.get('content', '')
                print(f"    Content:\n    {content}")
    else:
        print(f"\n WIKIPEDIA PASSAGES: None")
    
    print(f"\n{'='*80}")
    print(" FINAL ANSWER:")
    print(f"{'='*80}")
    print(result['answer'])
    
    print(f"\n{'='*80}")
    print(" FORMATTED CONTEXT (used for answer generation):")
    print(f"{'='*80}")
    print(result.get('context', 'None'))
    
else:
    print(f" Image not found: {test_image_path}")


Testing with image: 004160.jpg
Question: Địa điểm trong ảnh có phải di sản thiên nhiên thế giới không?

 Processing...


ERROR:src.vision:Error in OCR: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/vie.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'vie\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')



 DETAILED RESULTS

 CAPTION:
Hình ảnh này là một bức ảnh chụp từ trên cao, cho thấy một cảnh quan tự nhiên tuyệt đẹp với hàng ngàn hòn đảo nhỏ và lớn trải dài trên mặt nước. Đây là một khu vực có nhiều hòn đảo và núi đá vôi, tạo nên một cảnh quan độc đáo và huyền ảo. 

Cảnh quan này có nhiều màu sắc đa dạng, từ màu xanh lá cây của cây cỏ trên các hòn đảo, đến màu xanh ngắt của nước biển và màu trắng của các ngôi nhà nhỏ trên các hòn đảo. Có rất nhiều thuyền và tàu thuyền đang di chuyển trên mặt nước, tạo nên một cảnh quan sinh động và sôi động.

Cảnh quan này có thể là một phần của Vịnh Hạ Long, một di sản thế giới nổi tiếng ở Việt Nam. Vịnh Hạ Long nổi tiếng với hàng ngàn hòn đảo và núi đá vôi, tạo nên một cảnh quan tự nhiên tuyệt đẹp và độc đáo.

 OCR:


 RETRIEVAL SUMMARY:
  Total retrieved documents: 3
  - From Knowledge Base: 0
  - From Wikipedia: 3

 WIKIPEDIA SEARCH KEYWORDS (VLM-generated):
  Vịnh Hạ Long, di sản thiên nhiên thế giới, Việt Nam, hòn đảo, núi đá vôi.

 KNOWLEDGE